In [1]:
import pandas as pd
import numpy as np

In [ ]:
trn = pd.read_csv('./dataset/train_ver2.csv')

In [ ]:
prods = trn.columns[24:].tolist()

In [ ]:
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")]
    int_date = (int(Y) - 2015) * 12 + int(M)
    
    return int_date

trn['int_date'] = trn['fecha_dato'].map(date_to_int).astype(np.int8)

In [ ]:
trn_lag = trn.copy()

trn_lag['int_date'] += 1
trn_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in trn.columns]

del trn_lag['sexo_prev']
del trn_lag['fecha_alta_prev']
del trn_lag['canal_entrada_prev']

In [ ]:
trn.head()

In [ ]:
df_trn = trn.merge(trn_lag, on=['ncodpers', 'int_date'], how='left')

del trn, trn_lag

df_trn.head()

In [ ]:
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.head()

In [ ]:
for prod in prods:
    padd = prod + '_add'
    prev = prod + '_prev'
    df_trn[padd] = ((df_trn[prod] == 1) & (df_trn[prev] == 0)).astype(np.int8) 

In [35]:
add_cols = [prod + '_add' for prod in prods]
labels = df_trn[add_cols].copy()
labels.columns = prods

In [ ]:
labels.to_csv('./dataset/labels.csv', index=False)